# **Práctica #1: Modelo para predicción de lluvia**

*Centro Universitario de Ciencias Exactas e Ingenierías*

*División de Tecnologías para la Integración Ciber-Humana*

*Ingeniería Biomédica*

<br>

*Mtra. Sofía Alejandra Aguilar Valdez*

2 de septiembre de 2022

## **Información del equipo**

```NOMBRES:``` \\
Victor Alvarado Aparicio \\
Dayana Analy Pacheco Bañuelos \\
Leonardo Juàrez Zucco

```CÓDIGOS:``` \\
215767685 \\
######### \\
220285869

```LINK REPOSITORIO:``` https://github.com/L-Rittmeister/U1_Team.git



## **Contenido**



1.   Resumen
2.   Marco teórico
3.   Objetivos
4.   Materiales y métodos
5.   Resultados
6.   Discusión
7.   Conclusiones
8.   Referencias



# **1. Resumen**
(100-300 palabras)

# **2. Marco teórico**
(300-800 palabras)

# **3. Objetivos**
Objetivo general y objetivos específicos

# **4. Materiales y métodos**

## *Materiales*

Describir este conjunto de datos [[1]](https://www.kaggle.com/code/fatmakursun/rain-forecasting-with-artificial-neural-network/data).

## *Métodos*

1. Esquema de metodología
2. Descripción de los métodos y su implementación en código en forma de narrativa.

In [74]:
#Mounting drive storage to access data (if needed/running on collab then uncomment)
# Mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [75]:
# Import Libraries
import pandas as pd
import numpy as np
from numpy import sqrt
from numpy import vstack
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from tqdm.notebook import trange, tqdm
import time
import torch
from torch import nn
from torch.nn.functional import normalize
from torchvision import datasets, transforms
import torch.utils.data as data
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
from torch.optim import SGD
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from torch.nn.modules.activation import ReLU
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [76]:
#Pytorch and CUDA installation test
torch.randn(5).cuda()

tensor([-0.6030,  1.2772, -2.3043,  0.6307, -0.3543], device='cuda:0')

In [77]:
#Load the dataset on a pandas dataframe
Data = pd.read_csv("seattleWeather_1948-2017.csv")
Data

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
...,...,...,...,...,...
25546,2017-12-10,0.00,49,34,False
25547,2017-12-11,0.00,49,29,False
25548,2017-12-12,0.00,46,32,False
25549,2017-12-13,0.00,48,34,False


In [78]:
#Drop NaN value
Data.dropna(inplace=True)
Data.reset_index(inplace=True, drop=True)

In [79]:
#Create a n array with the values of PRCP, TMAX, TMIN, and RAIN
Data = Data[['PRCP', 'TMAX', 'TMIN', 'RAIN']]
Data

,PRCP,TMAX,TMIN,RAIN
0,0.47,51,42,True
1,0.59,45,36,True
2,0.42,45,35,True
3,0.31,45,34,True
4,0.17,45,32,True
...,...,...,...,...
25543,0.00,49,34,False
25544,0.00,49,29,False
25545,0.00,46,32,False
25546,0.00,48,34,False


In [80]:
#Eliminate possible void values
Data = Data[Data['PRCP'] != 'T']
Data = Data[Data['TMAX'] != 'T']
Data = Data[Data['TMIN'] != 'T']
Data = Data[Data['RAIN'] != 'T']
Data

,PRCP,TMAX,TMIN,RAIN
0,0.47,51,42,True
1,0.59,45,36,True
2,0.42,45,35,True
3,0.31,45,34,True
4,0.17,45,32,True
...,...,...,...,...
25543,0.00,49,34,False
25544,0.00,49,29,False
25545,0.00,46,32,False
25546,0.00,48,34,False


In [81]:
#Convert the values of PRCP, TMAX, TMIN, and RAIN to float
Data['PRCP'] = Data['PRCP'].astype(float)
Data['TMAX'] = Data['TMAX'].astype(float)
Data['TMIN'] = Data['TMIN'].astype(float)
Data['RAIN'] = Data['RAIN'].astype(float)
Data

,PRCP,TMAX,TMIN,RAIN
0,0.47,51.0,42.0,1.0
1,0.59,45.0,36.0,1.0
2,0.42,45.0,35.0,1.0
3,0.31,45.0,34.0,1.0
4,0.17,45.0,32.0,1.0
...,...,...,...,...
25543,0.00,49.0,34.0,0.0
25544,0.00,49.0,29.0,0.0
25545,0.00,46.0,32.0,0.0
25546,0.00,48.0,34.0,0.0


In [82]:
#Create the tensor
Data = torch.tensor(Data.values)

In [83]:
#Split the tensor into training and testing
Train = Data[:int(0.8*len(Data))]
Test = Data[int(0.8*len(Data)):]

In [84]:
#Create the training and testing dataloaders
Train_loader = DataLoader(Train, batch_size=2, shuffle=True)
Test_loader = DataLoader(Test, batch_size=2, shuffle=True)

In [85]:
#Create the model
class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()
        self.l1=nn.Linear(4, 4)
        self.l2=nn.Linear(4, 4)
        self.l3=nn.Linear(4, 1)
        self.do=nn.Dropout(0.1)
    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1))
        do = self.do(h2+h1)
        logits = self.l3(do)
        return logits

In [86]:
Model = Predictor().cuda()

In [87]:
#Define cost and optimizer
Parameters = Model.parameters()
Optimizer = torch.optim.SGD(Parameters, lr=0.001)
Loss = nn.CrossEntropyLoss()

In [88]:
#Greate target for the training according to 32 batch sizes
Target = torch.zeros(2, dtype=torch.long).cuda()

In [ ]:
#Train the model and calculate the accuracy
def train(Model, Train_loader,Target, Loss, Optimizer, Epochs):
    Model.train()
    for epoch in tqdm(range(Epochs)):
        for batch_idx, sample in tqdm(enumerate(Train_loader)):
            #print(sample)
            #data, target = sample['data'].cuda()#, sample['target'].cuda()
            data = sample.cuda()
            Optimizer.zero_grad()
            output = Model(data.float())
            loss = Loss(output, Target)
            loss.backward()
            Optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(Train_loader.dataset),
                    100. * batch_idx / len(Train_loader), loss.item()))

In [92]:
#Test the trainer
train(Model, Train_loader, Target, Loss, Optimizer, 5)

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Train Epoch: 0 [0/20438 (0%)]	Loss: 0.000000
Train Epoch: 0 [200/20438 (1%)]	Loss: 0.000000
Train Epoch: 0 [400/20438 (2%)]	Loss: 0.000000
Train Epoch: 0 [600/20438 (3%)]	Loss: 0.000000
Train Epoch: 0 [800/20438 (4%)]	Loss: 0.000000
Train Epoch: 0 [1000/20438 (5%)]	Loss: 0.000000
Train Epoch: 0 [1200/20438 (6%)]	Loss: 0.000000
Train Epoch: 0 [1400/20438 (7%)]	Loss: 0.000000
Train Epoch: 0 [1600/20438 (8%)]	Loss: 0.000000
Train Epoch: 0 [1800/20438 (9%)]	Loss: 0.000000
Train Epoch: 0 [2000/20438 (10%)]	Loss: 0.000000
Train Epoch: 0 [2200/20438 (11%)]	Loss: 0.000000
Train Epoch: 0 [2400/20438 (12%)]	Loss: 0.000000
Train Epoch: 0 [2600/20438 (13%)]	Loss: 0.000000
Train Epoch: 0 [2800/20438 (14%)]	Loss: 0.000000
Train Epoch: 0 [3000/20438 (15%)]	Loss: 0.000000
Train Epoch: 0 [3200/20438 (16%)]	Loss: 0.000000
Train Epoch: 0 [3400/20438 (17%)]	Loss: 0.000000
Train Epoch: 0 [3600/20438 (18%)]	Loss: 0.000000
Train Epoch: 0 [3800/20438 (19%)]	Loss: 0.000000
Train Epoch: 0 [4000/20438 (20%)]	Los

0it [00:00, ?it/s]

Train Epoch: 1 [0/20438 (0%)]	Loss: 0.000000
Train Epoch: 1 [200/20438 (1%)]	Loss: 0.000000
Train Epoch: 1 [400/20438 (2%)]	Loss: 0.000000
Train Epoch: 1 [600/20438 (3%)]	Loss: 0.000000
Train Epoch: 1 [800/20438 (4%)]	Loss: 0.000000
Train Epoch: 1 [1000/20438 (5%)]	Loss: 0.000000
Train Epoch: 1 [1200/20438 (6%)]	Loss: 0.000000
Train Epoch: 1 [1400/20438 (7%)]	Loss: 0.000000
Train Epoch: 1 [1600/20438 (8%)]	Loss: 0.000000
Train Epoch: 1 [1800/20438 (9%)]	Loss: 0.000000
Train Epoch: 1 [2000/20438 (10%)]	Loss: 0.000000
Train Epoch: 1 [2200/20438 (11%)]	Loss: 0.000000
Train Epoch: 1 [2400/20438 (12%)]	Loss: 0.000000
Train Epoch: 1 [2600/20438 (13%)]	Loss: 0.000000
Train Epoch: 1 [2800/20438 (14%)]	Loss: 0.000000
Train Epoch: 1 [3000/20438 (15%)]	Loss: 0.000000
Train Epoch: 1 [3200/20438 (16%)]	Loss: 0.000000
Train Epoch: 1 [3400/20438 (17%)]	Loss: 0.000000
Train Epoch: 1 [3600/20438 (18%)]	Loss: 0.000000
Train Epoch: 1 [3800/20438 (19%)]	Loss: 0.000000
Train Epoch: 1 [4000/20438 (20%)]	Los

...

Train Epoch: 2 [0/20438 (0%)]	Loss: 0.000000
Train Epoch: 2 [200/20438 (1%)]	Loss: 0.000000
Train Epoch: 2 [400/20438 (2%)]	Loss: 0.000000
Train Epoch: 2 [600/20438 (3%)]	Loss: 0.000000
Train Epoch: 2 [800/20438 (4%)]	Loss: 0.000000
Train Epoch: 2 [1000/20438 (5%)]	Loss: 0.000000
Train Epoch: 2 [1200/20438 (6%)]	Loss: 0.000000
Train Epoch: 2 [1400/20438 (7%)]	Loss: 0.000000
Train Epoch: 2 [1600/20438 (8%)]	Loss: 0.000000
Train Epoch: 2 [1800/20438 (9%)]	Loss: 0.000000
Train Epoch: 2 [2000/20438 (10%)]	Loss: 0.000000
Train Epoch: 2 [2200/20438 (11%)]	Loss: 0.000000
Train Epoch: 2 [2400/20438 (12%)]	Loss: 0.000000
Train Epoch: 2 [2600/20438 (13%)]	Loss: 0.000000
Train Epoch: 2 [2800/20438 (14%)]	Loss: 0.000000
Train Epoch: 2 [3000/20438 (15%)]	Loss: 0.000000
Train Epoch: 2 [3200/20438 (16%)]	Loss: 0.000000
Train Epoch: 2 [3400/20438 (17%)]	Loss: 0.000000
Train Epoch: 2 [3600/20438 (18%)]	Loss: 0.000000
Train Epoch: 2 [3800/20438 (19%)]	Loss: 0.000000
Train Epoch: 2 [4000/20438 (20%)]	Los

0it [00:00, ?it/s]

Train Epoch: 3 [0/20438 (0%)]	Loss: 0.000000
Train Epoch: 3 [200/20438 (1%)]	Loss: 0.000000
Train Epoch: 3 [400/20438 (2%)]	Loss: 0.000000
Train Epoch: 3 [600/20438 (3%)]	Loss: 0.000000
Train Epoch: 3 [800/20438 (4%)]	Loss: 0.000000
Train Epoch: 3 [1000/20438 (5%)]	Loss: 0.000000
Train Epoch: 3 [1200/20438 (6%)]	Loss: 0.000000
Train Epoch: 3 [1400/20438 (7%)]	Loss: 0.000000
Train Epoch: 3 [1600/20438 (8%)]	Loss: 0.000000
Train Epoch: 3 [1800/20438 (9%)]	Loss: 0.000000
Train Epoch: 3 [2000/20438 (10%)]	Loss: 0.000000
Train Epoch: 3 [2200/20438 (11%)]	Loss: 0.000000
Train Epoch: 3 [2400/20438 (12%)]	Loss: 0.000000
Train Epoch: 3 [2600/20438 (13%)]	Loss: 0.000000
Train Epoch: 3 [2800/20438 (14%)]	Loss: 0.000000
Train Epoch: 3 [3000/20438 (15%)]	Loss: 0.000000
Train Epoch: 3 [3200/20438 (16%)]	Loss: 0.000000
Train Epoch: 3 [3400/20438 (17%)]	Loss: 0.000000
Train Epoch: 3 [3600/20438 (18%)]	Loss: 0.000000
Train Epoch: 3 [3800/20438 (19%)]	Loss: 0.000000
Train Epoch: 3 [4000/20438 (20%)]	Los

0it [00:00, ?it/s]

Train Epoch: 4 [0/20438 (0%)]	Loss: 0.000000
Train Epoch: 4 [200/20438 (1%)]	Loss: 0.000000
Train Epoch: 4 [400/20438 (2%)]	Loss: 0.000000
Train Epoch: 4 [600/20438 (3%)]	Loss: 0.000000
Train Epoch: 4 [800/20438 (4%)]	Loss: 0.000000
Train Epoch: 4 [1000/20438 (5%)]	Loss: 0.000000
Train Epoch: 4 [1200/20438 (6%)]	Loss: 0.000000
Train Epoch: 4 [1400/20438 (7%)]	Loss: 0.000000
Train Epoch: 4 [1600/20438 (8%)]	Loss: 0.000000
Train Epoch: 4 [1800/20438 (9%)]	Loss: 0.000000
Train Epoch: 4 [2000/20438 (10%)]	Loss: 0.000000
Train Epoch: 4 [2200/20438 (11%)]	Loss: 0.000000
Train Epoch: 4 [2400/20438 (12%)]	Loss: 0.000000
Train Epoch: 4 [2600/20438 (13%)]	Loss: 0.000000
Train Epoch: 4 [2800/20438 (14%)]	Loss: 0.000000
Train Epoch: 4 [3000/20438 (15%)]	Loss: 0.000000
Train Epoch: 4 [3200/20438 (16%)]	Loss: 0.000000
Train Epoch: 4 [3400/20438 (17%)]	Loss: 0.000000
Train Epoch: 4 [3600/20438 (18%)]	Loss: 0.000000
Train Epoch: 4 [3800/20438 (19%)]	Loss: 0.000000
Train Epoch: 4 [4000/20438 (20%)]	Los

In [97]:
#Test the model
def test(Model, Test_loader, Loss):
    Model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for sample in tqdm(Test_loader):
            #data, target = sample['data'].cuda(), sample['target'].cuda()
            data = sample.cuda()
            output = Model(data.float())
            test_loss += Loss(output, Target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(Target.view_as(pred)).sum().item()

    test_loss /= len(Test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(Test_loader.dataset),
        100. * correct / len(Test_loader.dataset)))

In [98]:
#Test the tester
test(Model, Test_loader, Loss)

  0%|          | 0/2555 [00:00<?, ?it/s]


Test set: Average loss: 0.0000, Accuracy: 5110/5110 (100%)



# **5. Resultados**

# **6. Discusión**
Underfitting, overfitting, o no y el porqué
Comparación de su metodología con literatura asociada

# **7. Conclusiones**
(100-300 palabras).

# **8. Referencias**

[1] Pb, V. (2020, February 18). Perceptron. Kaggle. Retrieved June 1, 2022, from https://www.kaggle.com/code/prashfio/perceptron/notebook